# 0. Read Data

In [0]:
users = spark.read.format("delta").load("dbfs:/user/hive/warehouse/user_takehome")
products = spark.read.format("delta").load("dbfs:/user/hive/warehouse/products_takehome")
transactions = spark.read.format("delta").load("dbfs:/user/hive/warehouse/transaction_takehome")

# convert it to SQL
users.createOrReplaceTempView("users")
products.createOrReplaceTempView("products")
transactions.createOrReplaceTempView("transactions")

# convert it to Pandas dataframe
df_users = users.toPandas()
df_products = products.toPandas()
df_transactions = transactions.toPandas()

# 1. Explore the Data

## 1.1 General Missing Data Info

In [0]:
# Some users don't have complete demographic information
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   ID            100000 non-null  object        
 1   CREATED_DATE  100000 non-null  datetime64[ns]
 2   BIRTH_DATE    96325 non-null   datetime64[ns]
 3   STATE         95188 non-null   object        
 4   LANGUAGE      69492 non-null   object        
 5   GENDER        94108 non-null   object        
dtypes: datetime64[ns](2), object(4)
memory usage: 4.6+ MB


In [0]:
# Some products don't have Barcodes
# Some products have incomplete category, brand, and manufacturer information
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 845552 entries, 0 to 845551
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   CATEGORY_1    845441 non-null  object 
 1   CATEGORY_2    844128 non-null  object 
 2   CATEGORY_3    784986 non-null  object 
 3   CATEGORY_4    67459 non-null   object 
 4   MANUFACTURER  619078 non-null  object 
 5   BRAND         619080 non-null  object 
 6   BARCODE       841527 non-null  float64
dtypes: float64(1), object(6)
memory usage: 45.2+ MB


In [0]:
# Some transactions don't have Barcodes
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   RECEIPT_ID      50000 non-null  object        
 1   PURCHASE_DATE   50000 non-null  object        
 2   SCAN_DATE       50000 non-null  datetime64[ns]
 3   STORE_NAME      50000 non-null  object        
 4   USER_ID         50000 non-null  object        
 5   BARCODE         44238 non-null  float64       
 6   FINAL_QUANTITY  50000 non-null  object        
 7   FINAL_SALE      50000 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 3.1+ MB


## 1.2 Data type mismatch

In [0]:
# FINAL_QUANTITY is a string, but it should be an integer or decimal
# There is a value 'zero', which should be converted to '0'
df_transactions.FINAL_QUANTITY.unique()

array(['1', 'zero', '2', '3', '4', '4.55', '2.83', '2.34', '0.46', '7',
       '18', '12', '5', '2.17', '0.23', '8', '1.35', '0.09', '2.58',
       '1.47', '16', '0.62', '1.24', '1.4', '0.51', '0.53', '1.69', '6',
       '2.39', '2.6', '10', '0.86', '1.54', '1.88', '2.93', '1.28',
       '0.65', '2.89', '1.44', '2.75', '1.81', '276', '0.87', '2.1',
       '3.33', '2.54', '2.2', '1.93', '1.34', '1.13', '2.19', '0.83',
       '2.61', '0.28', '1.5', '0.97', '0.24', '1.18', '6.22', '1.22',
       '1.23', '2.57', '1.07', '2.11', '0.48', '9', '3.11', '1.08',
       '5.53', '1.89', '0.01', '2.18', '1.99', '0.04', '2.25', '1.37',
       '3.02', '0.35', '0.99', '1.8', '3.24', '0.94', '2.04', '3.69',
       '0.7', '2.52', '2.27'], dtype=object)

## 1.3 Columns (fields) are hard to understand

### 1.3.1 FINAL_SALE

In [0]:
%sql
-- Many receipts have duplicate records which have the same purchase date, scan date, store name, user id, barcode, final quantity, but different fianl sale (usually one has a value and the other does not.)
SELECT RECEIPT_ID, PURCHASE_DATE, SCAN_DATE, STORE_NAME, USER_ID, BARCODE, FINAL_QUANTITY, FINAL_SALE
FROM transactions
WHERE RECEIPT_ID in (
  SELECT RECEIPT_ID
  FROM transactions 
  GROUP BY RECEIPT_ID
  HAVING count(FINAL_QUANTITY) > 1
)
ORDER BY RECEIPT_ID
LIMIT 10

RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21,2024-08-21T14:19:06.539Z,WALMART,63b73a7f3d310dceeabd4758,15300014978,1,1.54
0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21,2024-08-21T14:19:06.539Z,WALMART,63b73a7f3d310dceeabd4758,15300014978,1,
0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20T09:50:24.206Z,ALDI,62c08877baa38d1a1f6c211a,null,1,1.49
0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20T09:50:24.206Z,ALDI,62c08877baa38d1a1f6c211a,null,zero,1.49
00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18,2024-08-19T15:38:56.813Z,WALMART,60842f207ac8b7729e472020,78742229751,1,
00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18,2024-08-19T15:38:56.813Z,WALMART,60842f207ac8b7729e472020,78742229751,1,2.54
000239aa-3478-453d-801e-66a82e39c8af,2024-06-18,2024-06-19T11:03:37.468Z,FOOD LION,63fcd7cea4f8442c3386b589,783400000000,1,3.49
000239aa-3478-453d-801e-66a82e39c8af,2024-06-18,2024-06-19T11:03:37.468Z,FOOD LION,63fcd7cea4f8442c3386b589,783400000000,zero,3.49
00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04,2024-07-05T15:56:43.549Z,RANDALLS,6193231ae9b3d75037b0f928,47900501183,1,5.29
00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04,2024-07-05T15:56:43.549Z,RANDALLS,6193231ae9b3d75037b0f928,47900501183,1,


### 1.3.2 Duplicate BARCODE with different BRAND name

In [0]:
%sql
-- Some BARCODEs have multiple BRAND names, and it will cause duplicate records when joining tables
SELECT *
FROM products 
WHERE BARCODE IN (
  SELECT BARCODE
  FROM products
  GROUP BY BARCODE
  HAVING count(BRAND) > 1
)

CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
Snacks,Candy,Chocolate Candy,null,THE HERSHEY COMPANY,MR. GOODBAR,3424407
Snacks,Candy,Mints,null,THE HERSHEY COMPANY,ICE BREAKERS,3468005
Health & Wellness,Skin Care,Lip Balms & Treatments,Medicated Lip Treatments,BLISTEX INC,BLISTEX,4138891
Snacks,Dessert Toppings,Ice Cream Sauces & Syrups,null,THE HERSHEY COMPANY,REESE'S,3408704
Snacks,Candy,Gum,null,THE HERSHEY COMPANY,ICE BREAKERS,3464502
Snacks,Candy,Gum,null,CHICLES CANELS SA DE CV,CANELS,75030452
Snacks,Candy,Mints,null,THE HERSHEY COMPANY,ICE BREAKERS,6811600
Snacks,Candy,Confection Candy,null,MARS WRIGLEY,STARBURST,400510
Snacks,Candy,Chocolate Candy,null,THE HERSHEY COMPANY,HERSHEY'S,3447109
Snacks,Candy,Chocolate Candy,null,THE HERSHEY COMPANY,REESE'S,3421109


## 1.4 Some Challenges after Joining Tables 
There are total 50,000 records in transaction table; however, due to some duplicate BARCODEs, there are more than 50,000 records (50,024) when joining three tables together. 

In [0]:
%sql
-- count of total transaction records
SELECT count(*)
FROM transactions

count(1)
50000


In [0]:
%sql
-- count of total transaction records after joining three tables
SELECT count(*)
FROM transactions a 
LEFT JOIN users b 
  ON a.USER_ID = b.ID
LEFT JOIN products c 
  ON a.BARCODE = c.BARCODE

count(1)
50024


# 2. SQL Queries

## 2.1 Closed-ended question
What are the top 5 brands by receipts scanned among users 21 and over?

In [0]:
%sql
SELECT BRAND, cnt, rnk 
FROM (
  SELECT c.BRAND, COUNT(RECEIPT_ID) cnt, DENSE_RANK()OVER(order by count(RECEIPT_ID) DESC) as rnk
  FROM transactions a 
  LEFT JOIN users b 
    on a.USER_ID = b.ID
  LEFT JOIN products c 
    on a.BARCODE = c.BARCODE
  WHERE datediff(current_date(), b.BIRTH_DATE) >= 21
  GROUP BY c.BRAND
)
WHERE rnk <= 5
ORDER BY rnk asc, cnt desc

BRAND,cnt,rnk
null,146,1
NERDS CANDY,6,2
DOVE,6,2
TRIDENT,4,3
GREAT VALUE,4,3
CHEX MIX,4,3
FRESHNESS GUARANTEED,2,4
L'OREAL PARIS HAIR CARE,2,4
BABY RUTH,2,4
DEAN'S DAIRY DIP,2,4


## 2.2 Closed-ended question
What are the top 5 brands by sales among users that have had their account for at least six months?

In [0]:
%sql
SELECT BRAND, total_sale, rnk 
FROM (
  SELECT c.BRAND, ROUND(SUM(FINAL_SALE), 3) total_sale, DENSE_RANK()OVER(order by SUM(FINAL_SALE) DESC) as rnk
  FROM transactions a 
  LEFT JOIN users b 
    on a.USER_ID = b.ID
  LEFT JOIN products c 
    on a.BARCODE = c.BARCODE
  WHERE datediff(month, b.CREATED_DATE, current_date()) >= 6
  GROUP BY c.BRAND
)
WHERE rnk <= 5
ORDER BY rnk asc

BRAND,total_sale,rnk
null,447.06,1
CVS,72.0,2
TRIDENT,46.72,3
DOVE,42.88,4
COORS LIGHT,34.96,5


## 2.3 Closed-ended question
What is the percentage of sales in the Health & Wellness category by generation?

In [0]:
%sql
WITH GenerationSales AS (
    SELECT 
        CASE
            WHEN YEAR(b.BIRTH_DATE) BETWEEN 1946 AND 1964 THEN 'Baby Boomer'
            WHEN YEAR(b.BIRTH_DATE) BETWEEN 1965 AND 1980 THEN 'Gen X'
            WHEN YEAR(b.BIRTH_DATE) BETWEEN 1981 AND 1996 THEN 'Millennial'
            WHEN YEAR(b.BIRTH_DATE) BETWEEN 1997 AND 2012 THEN 'Gen Z'
            ELSE 'Other'
        END AS Generation,
        SUM(a.FINAL_SALE) AS GenerationSales
    FROM transactions a 
    LEFT JOIN users b 
        ON a.USER_ID = b.ID
    LEFT JOIN products c 
        ON a.BARCODE = c.BARCODE
    WHERE c.CATEGORY_1 = 'Health & Wellness'
    GROUP BY Generation
),
TotalSales AS (
    SELECT SUM(a.FINAL_SALE) AS TotalSales
    FROM transactions a 
    LEFT JOIN users b 
        ON a.USER_ID = b.ID
    LEFT JOIN products c 
        ON a.BARCODE = c.BARCODE
    WHERE c.CATEGORY_1 = 'Health & Wellness'
)
SELECT 
    gs.Generation,
    round(gs.GenerationSales, 3) as GenerationSales,
    round(ts.TotalSales, 3) as TotalSales,
    round((gs.GenerationSales / ts.TotalSales) * 100, 3) AS SalesPercentage
FROM GenerationSales gs, TotalSales ts
ORDER BY gs.Generation;

Generation,GenerationSales,TotalSales,SalesPercentage
Baby Boomer,85.91,20494.96,0.419
Gen X,38.62,20494.96,0.188
Millennial,43.15,20494.96,0.211
Other,20327.28,20494.96,99.182


## 2.4 Open-ended question
Which is the leading brand in the Dips & Salsa category?
- Assume that we want to only use current existing data to determin which brand is the leading brand. 
- TOSTITOS is the leading brand in the Dips & Salsa category in 2024 based on its market share. 
- Its total category count is also the largest. 

In [0]:
%sql
WITH CTE AS (
  SELECT sum(FINAL_SALE) as total_sale
  FROM transactions a 
  LEFT JOIN users b 
    ON a.USER_ID = b.ID
  LEFT JOIN products c 
    ON a.BARCODE = c.BARCODE
  WHERE CATEGORY_2 = 'Dips & Salsa'
)

SELECT YEAR(PURCHASE_DATE) as year
  ,BRAND
  ,count(CATEGORY_3) as total_category
  ,round(sum(case when FINAL_QUANTITY = 'zero' then 0 else FINAL_QUANTITY end), 3) as total_sale_quantity
  ,round(sum(FINAL_SALE), 3) as total_sale
  ,round(sum(FINAL_SALE)/(select total_sale from CTE)*100.0, 3) as market_share
FROM transactions a 
LEFT JOIN users b 
    ON a.USER_ID = b.ID
LEFT JOIN products c 
    ON a.BARCODE = c.BARCODE
WHERE CATEGORY_2 = 'Dips & Salsa'
GROUP BY year, BRAND
ORDER BY market_share desc, total_sale desc, total_sale_quantity desc, total_category desc

year,BRAND,total_category,total_sale_quantity,total_sale,market_share
2024,TOSTITOS,66,56.0,239.54,18.962
2024,PACE,48,38.0,118.58,9.387
2024,FRITOS,38,33.0,91.73,7.261
2024,HELUVA GOOD!,30,20.0,85.46,6.765
2024,DEAN'S DAIRY DIP,34,22.0,67.97,5.381
2024,null,10,7.0,59.88,4.74
2024,HIDDEN VALLEY RANCH,18,13.0,59.42,4.704
2024,HIDDEN VALLEY,20,22.0,51.24,4.056
2024,HERDEZ,18,12.0,48.32,3.825
2024,PRIVATE LABEL,18,13.0,48.15,3.812


# 3. Communicate with Stakeholders

Construct an email or slack message that is understandable to a product or business leader who is not familiar with your day-to-day work. Summarize the results of your investigation. Include:
1. Key data quality issues and outstanding questions about the data
2. One interesting trend in the data
3. Use a finding from part 2 or come up with a new insight   
4. Request for action: explain what additional help, info, etc. you need to make sense of the data and resolve any outstanding issues

Hi XXX,

I wanted to share some key findings and observations from my recent investigation into our data. Below is a summary of the data quality issues we are currently facing, an interesting trend I discovered, and some action points that would help improve our data quality and potentially drive business decisions.

#### 1. Key Data Quality Issues and Outstanding Questions

There are several data quality challenges that I’ve identified in the **user**, **product**, and **transaction** tables:

- **Missing Records:**
  - We have numerous missing records across the users, product, and transaction tables, which are causing gaps in our data.
  
- **Issue 1 - Missing Barcode Information:**
  - Both the product and transaction tables have significant gaps in barcode information. This makes it difficult to link product details properly and track sales accurately.
  
- **Issue 2 - Duplicate Barcode with Different Brand Names:**
  - There are cases where the same barcode is associated with different brand names, leading to inconsistent data. This is a potential issue for accurate product tracking and reporting.
  
- **Issue 3 - Duplicate Records in `FINAL_SALE`:**
  - We also found instances of duplicate records where all fields are identical except for the `FINAL_SALE` values. This is likely to lead to inflated sales figures and errors in reporting.
  
**Outstanding Questions:**
- I’d like to understand how we can integrate the barcode data with the correct brand information to resolve the issue of mismatched barcodes.
- We need to discuss how we can build an automated script to detect and de-duplicate these duplicate records in the transaction table. This would help maintain consistent data quality and reduce errors moving forward.

#### 2. Interesting Trend in the Data

One interesting trend I uncovered involves **TOSTITO**, a brand within the Dips and Salsa category:

- **TOSTITO** leads with the most categories in this category, with **66 distinct categories**. This could present a strong opportunity for partnership, and we may want to explore deeper collaboration or promotional opportunities with them.

Additionally, I noticed that **some brands have more than 10 categories** in our data. We could explore partnerships with these brands to potentially increase sales and revenue.

#### 3. Request for Action

In order to address the data quality issues and maximize the potential from these insights, I would appreciate some help in the following areas:
- Clarifying how we can resolve the barcode-brand mismatches. Is there a specific source or process we can rely on to validate and integrate this data?
- Discussing how we can automate the de-duplication process for the transaction records. Would you be able to allocate resources or point me to any existing tools or scripts we can leverage?

These actions will help ensure that our data is accurate and can be used effectively to drive business decisions.

Looking forward to your thoughts and any further direction you can provide.

Best regards,    
En-Ning